In [25]:
!pip install spotipy

In [26]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [27]:
# Credenciales de la app de Spotify
CLIENT_ID = '04afb9a4dfc34a2ba6ab9fefc30358b5'
CLIENT_SECRET = '44ef46e579df499c8a155933264a04fb'

In [28]:
# Autenticación con Client Credentials Flow
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [29]:
# Buscar a Bad Bunny, imprimir si se encuentra
resultado = sp.search(q='Bad Bunny', type='artist', limit=1)
print(resultado['artists']['items'][0]['name'])

Bad Bunny


In [30]:
import pandas as pd
import numpy as np
from time import sleep

# Obtener el ID del artista
bad_bunny_id = resultado['artists']['items'][0]['id']

# Función para obtener todos los álbumes (de cualquier tipo)
def get_all_albums(artist_id, album_type='album'):
    albums = []
    results = sp.artist_albums(artist_id, album_type=album_type, limit=50)
    albums += results['items']
    while results['next']:
        results = sp.next(results)
        albums += results['items']
    return albums

# Obtener todos los álbumes y singles del artista
albums = get_all_albums(bad_bunny_id, album_type='album')
singles = get_all_albums(bad_bunny_id, album_type='single')
all_releases = albums + singles

# Crear sets de IDs para distinguir albums y singles
album_ids_set = {album['id'] for album in albums}
single_ids_set = {single['id'] for single in singles}

# Evitar duplicados
album_ids = list({album['id'] for album in all_releases})

# Extraer tracks de cada álbum/single
tracks_data = []

for album_id in album_ids:
    album = sp.album(album_id)
    release_date = album['release_date']
    album_name = album['name']

    # Detectar si es single
    if album_id in single_ids_set:
        album_name = 'Single'

    tracks = album['tracks']['items']
    
    for track in tracks:
        tracks_data.append({
            'track_name': track['name'],
            'track_id': track['id'],
            'album': album_name,
            'release_date': release_date,
            'duration_ms': track['duration_ms']
        })

# Convertimos a DataFrame
df_tracks = pd.DataFrame(tracks_data)

# Obtener popularidades en bloques de 50
popularities = []
track_ids = df_tracks['track_id'].tolist()

for i in range(0, len(track_ids), 50):
    batch = track_ids[i:i+50]
    try:
        track_infos = sp.tracks(batch)['tracks']
        popularities += [track['popularity'] for track in track_infos]
    except:
        popularities += [None] * len(batch)
        sleep(1)

# Añadir columnas al DataFrame
df_tracks['popularity'] = popularities
df_tracks['duration_min'] = df_tracks['duration_ms'] / 60000


df_tracks.head(50)


,track_name,track_id,album,release_date,duration_ms,popularity,duration_min
0,Dime,08wJw4VdkKImX4t1vL6MsO,Single,2018-03-23,284626,57,4.743767
1,CÓMO SE SIENTE - Remix,2tFwfmceQa1Y6nRPhYbEtC,Single,2020-05-09,227520,79,3.792000
2,De Museo,267NGliXM8YLVZiKAD9Otm,Single,2021-07-06,209770,64,3.496167
3,Loca - Remix,2ECIwi1a7mfokdDkkJ08Ne,Single,2018-03-16,346460,70,5.774333
4,Crecia,7GfSJBI4qbAG40RTa9jhVu,Single,2017-01-27,270000,57,4.500000
5,Báilame - Remix,4ocmZDCb43B5WRYMKZSxhM,Single,2017-07-31,217080,51,3.618000
6,Subimos de Rango,4yZI69ElgxddAUGNJjzc0j,Single,2019-08-21,266091,54,4.434850
7,ADIVINO,0LZy30mVmxqUpdQmaXKXBd,Single,2024-04-25,278644,76,4.644067
8,EL MUNDO ES MÍO,36DHxTW2xdr9GG15T9oK9L,EL ÚLTIMO TOUR DEL MUNDO,2020-11-27,165199,63,2.753317
9,TE MUDASTE,5RubKOuDoPn5Kj5TLVxSxY,EL ÚLTIMO TOUR DEL MUNDO,2020-11-27,130013,74,2.166883


In [31]:
len(df_tracks)

222

In [32]:
df_tracks['release_date'] = pd.to_datetime(df_tracks['release_date'])
df_tracks['release_date']

0     2018-03-23
1     2020-05-09
2     2021-07-06
3     2018-03-16
4     2017-01-27
         ...    
217   2025-01-05
218   2019-05-17
219   2024-12-05
220   2017-12-01
221   2017-04-07
Name: release_date, Length: 222, dtype: datetime64[ns]

In [33]:
grouped_df = df_tracks.groupby('album').agg(avg_popularity=('popularity', np.mean))
grouped_df.head(100)

,avg_popularity
album,
DeBÍ TiRAR MáS FOToS,88.529412
EL ÚLTIMO TOUR DEL MUNDO,69.625000
LAS QUE NO IBAN A SALIR,63.200000
OASIS,70.750000
Single,63.340659
Un Verano Sin Ti,79.478261
X 100PRE,68.800000
YHLQMDLG,72.650000
nadie sabe lo que va a pasar mañana,69.590909


In [34]:
df_tracks[df_tracks['track_name'].str.contains('una velita', case=False, na=False)]


,track_name,track_id,album,release_date,duration_ms,popularity,duration_min
51,Una Velita,059qzhNaJb8tsqpdl2bHfF,Single,2024-09-19,239986,66,3.999767


In [35]:
df_tracks[df_tracks['track_name'].str.contains('where she goes', case=False, na=False)]

,track_name,track_id,album,release_date,duration_ms,popularity,duration_min
104,WHERE SHE GOES,2sTDlCxmuZCTDKKk9f1qus,nadie sabe lo que va a pasar mañana,2023-10-13,231704,72,3.861733
138,WHERE SHE GOES,7ro0hRteUMfnOioTFI5TG1,Single,2023-05-18,231704,78,3.861733


In [36]:
df_tracks[df_tracks['track_name'].str.contains('amorfoda', case=False, na=False)]

,track_name,track_id,album,release_date,duration_ms,popularity,duration_min
113,Amorfoda,68QcquxCS39cQGTsRrJWIR,Single,2018-02-15,153887,76,2.564783


In [37]:
df_tracks[df_tracks['album']=='Single']

,track_name,track_id,album,release_date,duration_ms,popularity,duration_min
0,Dime,08wJw4VdkKImX4t1vL6MsO,Single,2018-03-23,284626,57,4.743767
1,CÓMO SE SIENTE - Remix,2tFwfmceQa1Y6nRPhYbEtC,Single,2020-05-09,227520,79,3.792000
2,De Museo,267NGliXM8YLVZiKAD9Otm,Single,2021-07-06,209770,64,3.496167
3,Loca - Remix,2ECIwi1a7mfokdDkkJ08Ne,Single,2018-03-16,346460,70,5.774333
4,Crecia,7GfSJBI4qbAG40RTa9jhVu,Single,2017-01-27,270000,57,4.500000
...,...,...,...,...,...,...,...
200,un x100to,6pD0ufEQq0xdHSsRbg9LBK,Single,2023-04-17,194563,79,3.242717
218,No Me Conoce - Remix,6Mgg57b5K2uwIwuTRuaBWE,Single,2019-05-17,309546,49,5.159100
219,EL CLúB,3zOgFY5kpQ6p2cwPIquZks,Single,2024-12-05,222568,73,3.709467
220,Chambea,1Bqg4yFeVDJxchh6MjkGKy,Single,2017-12-01,192001,68,3.200017


In [38]:
df_tracks[['track_name']].drop_duplicates().to_excel('canciones_bb.xlsx', index=False)


In [39]:
df_labels = pd.read_excel('canciones_bb_filled.xlsx')
df_tracks = df_tracks.merge(df_labels, on='track_name', how='left')
df_tracks.head()

,track_name,track_id,album,release_date,duration_ms,popularity,duration_min,is_reggaeton
0,Dime,08wJw4VdkKImX4t1vL6MsO,Single,2018-03-23,284626,57,4.743767,0
1,CÓMO SE SIENTE - Remix,2tFwfmceQa1Y6nRPhYbEtC,Single,2020-05-09,227520,79,3.792000,1
2,De Museo,267NGliXM8YLVZiKAD9Otm,Single,2021-07-06,209770,64,3.496167,0
3,Loca - Remix,2ECIwi1a7mfokdDkkJ08Ne,Single,2018-03-16,346460,70,5.774333,0
4,Crecia,7GfSJBI4qbAG40RTa9jhVu,Single,2017-01-27,270000,57,4.500000,0


In [40]:
correlation = df_tracks['popularity'].corr(df_tracks['is_reggaeton'])
print(correlation)

0.29058291228827543


In [41]:
df_tracks['is_reggaeton'] = df_tracks['is_reggaeton'].replace({
    1: 'is reggaeton',
    0: 'is not reggaeton'
})
df_tracks.groupby('is_reggaeton').agg(
    avg_popularity=('popularity', np.mean),
    track_count=('track_name', 'count'))

,avg_popularity,track_count
is_reggaeton,,
is not reggaeton,66.733813,139
is reggaeton,74.084337,83
